In [1]:
!pip install textattack

     |████████████████████████████████| 358kB 4.7MB/s 
     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 102kB 4.4MB/s 
     |████████████████████████████████| 337kB 5.8MB/s 
     |████████████████████████████████| 2.1MB 8.0MB/s 
     |████████████████████████████████| 225kB 18.1MB/s 
     |████████████████████████████████| 778kB 20.9MB/s 
     |████████████████████████████████| 20.1MB 18.6MB/s 
     |████████████████████████████████| 1.2MB 36.5MB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 798kB 36.5MB/s 
     |████████████████████████████████| 983kB 38.4MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 3.3MB 35.4MB/s 
     |████████████████████████████████| 901kB 35.3MB/s 
     |████████████████████████████████| 112kB 42.3MB/s 
     |████████████████████████████████| 245kB 39.7MB/s 
  Created wheel for lru-dict: filename=lru_dict-1.1.7-c

In [2]:
!pip install torchfile

  Created wheel for torchfile: filename=torchfile-0.1.0-cp37-none-any.whl size=5713 sha256=38eab0a842632c792d13a30fa547225c74565e77e2b0f395406377076e06f0b0
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchfile


In [3]:
!pip install -U gensim==4.0.0

     |████████████████████████████████| 23.9MB 130kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [4]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
from gensim.models import Phrases
from gensim.test.utils import get_tmpfile
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:36<00:00, 13.0MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpml9z_v3q.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [5]:
from textattack.transformations import WordSwap
from textattack.search_methods import GreedySearch, GreedyWordSwapWIR
from textattack.constraints.overlap import LevenshteinEditDistance
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML

In [6]:
from textattack.constraints.overlap import LevenshteinEditDistance
from textattack.constraints.pre_transformation import (RepeatModification,StopwordModification,MinWordLength,)
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.shared.attack import Attack
from textattack.transformations import (CompositeTransformation,
    WordSwapNeighboringCharacterSwap,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
    WordSwapRandomCharacterSubstitution,
    WordSwapQWERTY
)
from textattack.constraints.grammaticality.language_models import (
    LearningToWriteLanguageModel,
)
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.search_methods import AlzantotGeneticAlgorithm

from textattack.transformations import WordSwapEmbedding

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
corpus = api.load('text8')

[==================================================] 100.0% 31.6/31.6MB downloaded


In [9]:
fname1 = get_tmpfile("/content/drive/MyDrive/models/fasttext.model")
model1=FastText.load(fname1)
fname2 = get_tmpfile("/content/drive/MyDrive/models/word2vec.model")
model2=FastText.load(fname2)

In [10]:
def attack_stats(attack, dataset, num_of_examples=25):
  num_examples = num_of_examples
  num_remaining_attacks = num_examples
  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

  worklist = deque(range(0, num_examples))
  worklist_tail = worklist[-1]

  attack_log_manager = textattack.loggers.AttackLogManager()

  load_time = time.time()
          
  num_results = 0
  num_failures = 0
  num_successes = 0
  for result in attack.attack_dataset(dataset, indices=worklist):
      result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
      display(HTML(result_html_str))
      attack_log_manager.log_result(result)
              
      if not isinstance(result, textattack.attack_results.SkippedAttackResult):
          pbar.update(1)
      else:
          worklist_tail += 1
          pbar.update(1)
          worklist.append(worklist_tail)

      num_results += 1

      if (
          type(result) == textattack.attack_results.SuccessfulAttackResult
          or type(result) == textattack.attack_results.MaximizedAttackResult
      ):
          num_successes += 1
                  
      if type(result) == textattack.attack_results.FailedAttackResult:
          num_failures += 1
      pbar.set_description(
          "[Succeeded / Failed / Total] {} / {} / {}".format(
              num_successes, num_failures, num_results
          )
      )

  pbar.close()

  attack_log_manager.enable_stdout()
  attack_log_manager.log_summary()
  attack_log_manager.flush()
          
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")


In [11]:
from textattack.transformations import WordSwap
#using fasttext similar word

class Swapper(WordSwap): # For fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

### textattack/albert-base-v2-yelp-polarity

In [12]:
model_transformer = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-yelp-polarity")
tokenizer = AutoTokenizer("textattack/albert-base-v2-yelp-polarity")

model_wrapper = HuggingFaceModelWrapper(model_transformer, tokenizer)

In [13]:
# We're going to use our word swap class as the attack transformation.
transformation = CompositeTransformation(
            [
                WordSwapNeighboringCharacterSwap(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterDeletion(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterInsertion(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapQWERTY(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
            ]
        )
max_num_word_swaps=1
constraints = [
    MinWordLength(min_length=4),
    StopwordModification(),
    MaxWordsPerturbed(max_num_words=max_num_word_swaps),
    RepeatModification(),
]
# untargeted attack
goal_function = UntargetedClassification(model_wrapper)
search_method = GreedySearch()

attack= Attack(goal_function, constraints, transformation, search_method)

dataset =HuggingFaceDataset("yelp_polarity", None, "test")
print(attack)
attack_stats(attack, dataset,5)


textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


textattack: Loading datasets dataset yelp_polarity, split test.


Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/591dc79cb354c6f051b5bf61c77a446205c98156e01669c6218c9bd6aa15f581. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  False
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  False
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  False
      )
    (3): WordSwapQWERTY
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  1
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
)


...

[Succeeded / Failed / Total] 1 / 2 / 3:  60%|██████    | 3/5 [01:37<01:05, 32.62s/it]

[Succeeded / Failed / Total] 2 / 2 / 4:  80%|████████  | 4/5 [02:05<00:31, 31.43s/it]

[Succeeded / Failed / Total] 2 / 3 / 5: 100%|██████████| 5/5 [03:27<00:00, 41.41s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 60.0%  |
| Attack success rate:          | 40.0%  |
| Average perturbed word %:     | 4.28%  |
| Average num. words per input: | 80.4   |
| Avg num queries:              | 1145.6 |
+-------------------------------+--------+


textattack: Attack time: 207.0401484966278s


### 10 example

In [14]:
attack_stats(attack, dataset,10)

  0%|          | 0/10 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 1 / 1:  10%|█         | 1/10 [00:58<08:47, 58.62s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:  20%|██        | 2/10 [01:27<05:48, 43.61s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  30%|███       | 3/10 [01:37<03:46, 32.38s/it]

[Succeeded / Failed / Total] 2 / 2 / 4:  40%|████      | 4/10 [02:04<03:07, 31.25s/it]

...

[Succeeded / Failed / Total] 2 / 6 / 8:  80%|████████  | 8/10 [07:56<01:59, 59.57s/it]

[Succeeded / Failed / Total] 2 / 7 / 9:  90%|█████████ | 9/10 [09:05<01:00, 60.64s/it]

[Succeeded / Failed / Total] 2 / 8 / 10: 100%|██████████| 10/10 [09:37<00:00, 57.79s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 8      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 80.0%  |
| Attack success rate:          | 20.0%  |
| Average perturbed word %:     | 4.28%  |
| Average num. words per input: | 121.2  |
| Avg num queries:              | 1539.0 |
+-------------------------------+--------+


textattack: Attack time: 577.8765642642975s


### 15 example

In [15]:
attack_stats(attack, dataset,15)

  0%|          | 0/15 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 1 / 1:   7%|▋         | 1/15 [00:58<13:37, 58.40s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:  13%|█▎        | 2/15 [01:26<09:25, 43.47s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  20%|██        | 3/15 [01:36<06:27, 32.29s/it]

[Succeeded / Failed / Total] 2 / 2 / 4:  27%|██▋       | 4/15 [02:04<05:43, 31.22s/it]

...

[Succeeded / Failed / Total] 2 / 11 / 13:  87%|████████▋ | 13/15 [14:14<02:11, 65.73s/it]

[Succeeded / Failed / Total] 2 / 12 / 14:  93%|█████████▎| 14/15 [15:15<01:05, 65.37s/it]

[Succeeded / Failed / Total] 2 / 13 / 15: 100%|██████████| 15/15 [15:35<00:00, 62.37s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 13     |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 86.67% |
| Attack success rate:          | 13.33% |
| Average perturbed word %:     | 4.28%  |
| Average num. words per input: | 130.07 |
| Avg num queries:              | 1665.8 |
+-------------------------------+--------+


textattack: Attack time: 935.5097765922546s


### 20 example

In [16]:
attack_stats(attack, dataset,20)

  0%|          | 0/20 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 1 / 1:   5%|▌         | 1/20 [00:58<18:30, 58.44s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:  10%|█         | 2/20 [01:26<13:02, 43.48s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  15%|█▌        | 3/20 [01:37<09:09, 32.34s/it]

[Succeeded / Failed / Total] 2 / 2 / 4:  20%|██        | 4/20 [02:04<08:19, 31.20s/it]

...

[Succeeded / Failed / Total] 2 / 16 / 18:  90%|█████████ | 18/20 [19:53<02:12, 66.30s/it]

[Succeeded / Failed / Total] 2 / 17 / 19:  95%|█████████▌| 19/20 [21:36<01:08, 68.24s/it]

[Succeeded / Failed / Total] 2 / 18 / 20: 100%|██████████| 20/20 [22:07<00:00, 66.39s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 18     |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 90.0%  |
| Attack success rate:          | 10.0%  |
| Average perturbed word %:     | 4.28%  |
| Average num. words per input: | 140.0  |
| Avg num queries:              | 1736.2 |
+-------------------------------+--------+


textattack: Attack time: 1327.7061576843262s


### 30 example

In [17]:
attack_stats(attack, dataset,30)

  0%|          | 0/30 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 1 / 1:   3%|▎         | 1/30 [00:58<28:21, 58.66s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   7%|▋         | 2/30 [01:27<20:21, 43.62s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  10%|█         | 3/30 [01:37<14:34, 32.39s/it]

[Succeeded / Failed / Total] 2 / 2 / 4:  13%|█▎        | 4/30 [02:04<13:32, 31.23s/it]

...

[Succeeded / Failed / Total] 4 / 24 / 28:  93%|█████████▎| 28/30 [31:39<02:15, 67.86s/it]

[Succeeded / Failed / Total] 4 / 25 / 29:  97%|█████████▋| 29/30 [31:50<01:05, 65.87s/it]

[Succeeded / Failed / Total] 4 / 26 / 30: 100%|██████████| 30/30 [32:32<00:00, 65.09s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 26     |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 86.67% |
| Attack success rate:          | 13.33% |
| Average perturbed word %:     | 2.43%  |
| Average num. words per input: | 136.93 |
| Avg num queries:              | 1720.6 |
+-------------------------------+--------+


textattack: Attack time: 1952.8206379413605s
